<a href="https://colab.research.google.com/github/dtarrio/captura-y-almacenamiento-de-la-informacion/blob/main/TP1DiegoTarrio2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar los módulos necesarios
from urllib.request import Request, urlopen, build_opener, install_opener, ProxyHandler
import ssl
import json
import xml.etree.ElementTree as ET

# Para evitar errores de certificados inexistentes en algunas páginas web
ssl._create_default_https_context = ssl._create_unverified_context


In [3]:
#Especialización en Inteligencia de Datos orientado a BigData
#Materia: Catura y almacenamiento de la información
#año: 2024
#Alumno: Diego Fernando Tarrío (DNI: 22151919)
#Trabajo Práctico 1
#se extrae información de las guitarras marca Soloking que se encuentran publicadas en el sitio de
# ------  www.kaironmusic.com.ar ---------
#se extrae del HTML el listado de modelos de guitarra junto con el precio y se guardan en un archivo guitarras.json

# Se crea la petición
request = Request(
    url='https://www.kaironmusic.com.ar/search/?q=soloking&mpage=2',
    headers={'User-Agent': 'Mozilla/5.0'}
)

# Se realiza la descarga y se guarda el contenido HTML en una variable
page = urlopen(request)
content = page.read().decode("utf-8")

#analizado el html, se detectó que los modelos junto con la info de precios se guarda en
tag = "\"item_id\""
tagFin = "\"item_list_id\""

guitarras = []

content2 = content
start = content2.find(tag)
while start > 0:
  start = start + len(tag)
  content2 = content2[start:]
  end = content2.find(tagFin)

  data = content2[0: end]

  # Limpieza para quedarme sólo con el nombre y modelo de la guitarra marca Soloking
  start2 = data.find("Soloking")
  end2 = data.find("\"item_variant\"") - 2
  dataModelo = data[start2:end2].strip()

  # Limpieza para quedarme sólo con el precio de la guitarra
  start3 = data.find("\"price\":") + 8
  end3 = data.find("\"item_category2\"") - 1
  dataPrecio = data[start3:end3].strip()

  # Cambio el formato del precio para que sea más legible
  precioNumerico = float(dataPrecio)
  dataPrecio = "{:,.2f}".format(precioNumerico)

  #genero un diccionario con tuplas {Model,Price} por cada modelo encontrado
  guitarras.append({"Model":dataModelo,"Price": dataPrecio})

  start = content2.find(tag)

print(guitarras)

#guardo la lista de nombres y modelos de guitarras en un archivo con formato JSon
with open("guitarras.json","w") as file:
  json.dump(guitarras,file,indent=4)

#genero y guardo la misma información pero ahora en formato XML
root = ET.Element("Guitarras")
for guitarra in guitarras:
  model = ET.SubElement(root, "Model")
  model.text = guitarra["Model"]
  price = ET.SubElement(root, "Price")
  price.text = guitarra["Price"]

#creo el arbol XML y se guarda en archivo
tree = ET.ElementTree(root)
with open("guitarras.xml", "wb") as file:
  tree.write(file, encoding="utf-8", xml_declaration=True)

[]
